In [13]:
import json
import os
from typing import List, Dict, Any

# Configuración
STRAICO_DIR = ''
OUTPUT_DIR = 'Clean'
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Cargar modelos desde el archivo text_model_ids.txt
MODELS_FILE = 'models.txt'

def load_models_from_file(filepath):
    """Carga los nombres de modelos desde un archivo de texto."""
    try:
        with open(filepath, 'r', encoding='utf-8') as f:
            return [line.strip() for line in f if line.strip()]
    except FileNotFoundError:
        print(f"ERROR: No se encontró el archivo {filepath}")
        return []

DEFAULT_MODELS = load_models_from_file(MODELS_FILE)

print(f"Carpeta de entrada: {STRAICO_DIR}/")
print(f"Carpeta de salida: {OUTPUT_DIR}/")
print(f"Modelos: {DEFAULT_MODELS}")

Carpeta de entrada: /
Carpeta de salida: Clean/
Modelos: ['amazon/nova-micro-v1', 'microsoft/phi-4', 'amazon/nova-lite-v1', 'cohere/command-r-08-2024', 'qwen/qwen-2.5-72b-instruct', 'google/gemma-2-27b-it', 'meta-llama/llama-3.3-70b-instruct', 'microsoft/wizardlm-2-8x22b', 'meta-llama/llama-4-maverick', 'qwen/qwen2.5-vl-32b-instruct:free', 'x-ai/grok-3-mini-beta', 'perplexity/sonar', 'mistralai/mistral-medium-3', 'mistralai/mixtral-8x7b-instruct', 'google/gemini-2.5-flash', 'meta-llama/llama-3.1-405b-instruct', 'deepseek/deepseek-chat-v3.1', 'moonshotai/kimi-k2-0905', 'openai/o4-mini-high', 'openai/gpt-4.1', 'openai/o1-mini', 'anthropic/claude-sonnet-4', 'gpt-5.1']


## Cargar Dataset Original (Ground Truth)

In [14]:
import json

def load_ground_truth(json_path: str) -> Dict[str, Dict[str, str]]:
    """Carga el dataset original y elimina duplicados.
    
    Returns:
        Dict con modismo como clave y dict con {significado, ejemplo} como valor
    """
    ground_truth = {}
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
        
        # Si es una lista de objetos
        if isinstance(data, list):
            for item in data:
                modismo = (item.get('modismo') or '').strip()
                if modismo and modismo not in ground_truth:
                    ground_truth[modismo] = {
                        'significado': (item.get('significado') or '').strip(),
                        'ejemplo': (item.get('ejemplo') or '').strip()
                    }
        # Si es un dict directo
        elif isinstance(data, dict):
            for modismo, value in data.items():
                modismo = modismo.strip()
                if modismo and modismo not in ground_truth:
                    ground_truth[modismo] = {
                        'significado': (value.get('significado') or '').strip(),
                        'ejemplo': (value.get('ejemplo') or '').strip()
                    }
    
    print(f"Dataset cargado: {len(ground_truth)} modismos únicos")
    return ground_truth

# Cargar dataset
ground_truth = load_ground_truth('../DataSet/DataSet_PrimeraOcurrencia.json')

Dataset cargado: 6533 modismos únicos


## Funciones Auxiliares

In [15]:
def load_prompt_responses(prompt_dir: str) -> Dict[str, List[Dict]]:
    """Carga las respuestas de todos los modelos para un prompt.
    
    Args:
        prompt_dir: Directorio del prompt (ej: 'Straico/Prompt 1')
        
    Returns:
        Dict con modelo como clave y lista de respuestas como valor
    """

    all_models_path = os.path.join(prompt_dir, 'all_models.json')
    
    if os.path.exists(all_models_path):
        with open(all_models_path, 'r', encoding='utf-8') as f:
            return json.load(f)
    
    return {}


def extract_output_field(response: Dict, field: str) -> str:
    """Extrae un campo del output de la respuesta del modelo.
    
    Maneja múltiples formatos:
    1. Formato directo: {"output": {"campo": "valor"}}
    2. Formato raw_response: {"raw_response": "```json\\n{...}\\n```"}
    3. Errores: {"error": "..."}
    
    Args:
        response: Dict con la respuesta completa
        field: Campo a extraer del output
        
    Returns:
        Valor del campo como string, vacío si hay error o no se encuentra
    """
    try:
        if not isinstance(response, dict):
            return ''
        
        # Verificar si hay error
        if 'error' in response:
            return ''
        
        # Caso 1: Formato directo con output
        output = response.get('output', {})
        if isinstance(output, dict) and field in output:
            return str(output.get(field, '')).strip()
        
        # Caso 2: raw_response con JSON anidado
        raw_response = response.get('raw_response', '')
        if raw_response:
            # Limpiar markdown code blocks
            raw_response = raw_response.strip()
            if raw_response.startswith('```json'):
                raw_response = raw_response[7:]  # Remover ```json
            if raw_response.startswith('```'):
                raw_response = raw_response[3:]  # Remover ```
            if raw_response.endswith('```'):
                raw_response = raw_response[:-3]  # Remover ```
            
            raw_response = raw_response.strip()
            
            # Parsear el JSON anidado
            try:
                parsed = json.loads(raw_response)
                if isinstance(parsed, dict):
                    output = parsed.get('output', {})
                    if isinstance(output, dict) and field in output:
                        return str(output.get(field, '')).strip()
            except json.JSONDecodeError:
                return ''
        
        return ''
    except Exception as e:
        print(f"⚠ Error extrayendo campo '{field}': {e}")
        return ''


def save_json(filepath: str, data: Any):
    """Guarda datos en formato JSON."""
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
    print(f"✓ Guardado: {filepath} ({len(data)} registros)")

## PROMPT 1: Modismo → Es Modismo (Sí/No)

Procesa las respuestas del Prompt 1 para evaluar si los modelos identifican correctamente los modismos.

In [16]:
print("=" * 60)
print("PROCESANDO PROMPT 1: Modismo → Es Modismo (Sí/No)")
print("=" * 60)

prompt_1_responses = load_prompt_responses(os.path.join(STRAICO_DIR, 'Prompt 1'))

# Procesar datos
prompt_1_data = []
errors_count = 0
skipped_models = []
excluded_models = []  # Modelos excluidos por alto % de errores

for model in DEFAULT_MODELS:
    if model not in prompt_1_responses:
        print(f"⚠ Advertencia: Modelo {model} no encontrado en Prompt 1")
        skipped_models.append(model)
        continue
    
    model_responses = prompt_1_responses[model]
    total_entries = len(model_responses)
    model_errors = 0
    model_data = []  # Datos temporales del modelo
    
    for entry in model_responses:
        modismo = entry.get('modismo', '')
        response = entry.get('response', {})
        
        # Verificar si hay error en la respuesta
        if 'error' in response:
            model_errors += 1
            continue
        
        # Extraer respuesta (Sí/No) del output
        es_modismo_generado = extract_output_field(response, 'es_modismo')
        
        # Solo agregar si se extrajo la respuesta correctamente
        if es_modismo_generado:
            model_data.append({
                'modismo': modismo,
                'es_modismo_real': 'Sí',  # Todos los del dataset son modismos
                'modelo': model,
                'es_modismo_generado': es_modismo_generado
            })
        else:
            model_errors += 1
    
    # Calcular porcentaje de errores
    error_percentage = (model_errors / total_entries * 100) if total_entries > 0 else 0
    
    # Solo incluir si el porcentaje de errores es <= 50%
    if error_percentage > 50:
        print(f"  {model}: EXCLUIDO ({model_errors}/{total_entries} errores = {error_percentage:.1f}%)")
        excluded_models.append(model)
        errors_count += model_errors
    else:
        # Agregar datos del modelo
        prompt_1_data.extend(model_data)
        errors_count += model_errors
        if model_errors > 0:
            print(f"  {model}: {model_errors} errores omitidos ({error_percentage:.1f}%)")

# Guardar
output_path = os.path.join(OUTPUT_DIR, 'prompt_1_metrics_data.json')
save_json(output_path, prompt_1_data)

print(f"\n✓ Procesados {len(prompt_1_data)} registros válidos")
print(f"✗ Omitidos {errors_count} registros con errores")
if excluded_models:
    print(f"⊗ Modelos excluidos (>50% errores): {', '.join(excluded_models)}")
if skipped_models:
    print(f"⚠ Modelos no encontrados: {', '.join(skipped_models)}")
print()


PROCESANDO PROMPT 1: Modismo → Es Modismo (Sí/No)
  amazon/nova-micro-v1: 1 errores omitidos (0.0%)
  microsoft/phi-4: 10 errores omitidos (0.2%)
  amazon/nova-lite-v1: 3 errores omitidos (0.0%)
  cohere/command-r-08-2024: 3 errores omitidos (0.0%)
  qwen/qwen-2.5-72b-instruct: 708 errores omitidos (10.8%)
  google/gemma-2-27b-it: 15 errores omitidos (0.2%)
  meta-llama/llama-3.3-70b-instruct: 390 errores omitidos (6.0%)
  microsoft/wizardlm-2-8x22b: 184 errores omitidos (2.8%)
  meta-llama/llama-4-maverick: 8 errores omitidos (0.1%)
  qwen/qwen2.5-vl-32b-instruct:free: EXCLUIDO (6533/6533 errores = 100.0%)
  x-ai/grok-3-mini-beta: 284 errores omitidos (4.3%)
  perplexity/sonar: 279 errores omitidos (4.3%)
  mistralai/mistral-medium-3: 279 errores omitidos (4.3%)
  mistralai/mixtral-8x7b-instruct: 610 errores omitidos (9.3%)
  google/gemini-2.5-flash: 283 errores omitidos (4.3%)
  meta-llama/llama-3.1-405b-instruct: 300 errores omitidos (4.6%)
  deepseek/deepseek-chat-v3.1: 23 errores 

In [17]:
import pandas as pd

# Tabla de resultados por modelo para Prompt 1
prompt_1_stats = []

for model in DEFAULT_MODELS:
    if model not in prompt_1_responses:
        continue
    
    model_responses = prompt_1_responses[model]
    total_entries = len(model_responses)
    model_errors = 0
    valid_entries = 0
    
    for entry in model_responses:
        response = entry.get('response', {})
        
        if 'error' in response:
            model_errors += 1
            continue
        
        es_modismo_generado = extract_output_field(response, 'es_modismo')
        
        if es_modismo_generado:
            valid_entries += 1
        else:
            model_errors += 1
    
    prompt_1_stats.append({
        'Modelo': model,
        'Errores': model_errors,
        'Válidos': valid_entries,
        'Válidos/Total': f"{valid_entries}/{total_entries}",
        '% Válidos': f"{(valid_entries/total_entries*100):.1f}%" if total_entries > 0 else "0.0%"
    })

# Crear DataFrame y mostrar
df_prompt_1 = pd.DataFrame(prompt_1_stats)
print("\nTABLA PROMPT 1: Modismo → Es Modismo (Sí/No)")
print("=" * 80)
print(df_prompt_1.to_string(index=False))
print("=" * 80)


TABLA PROMPT 1: Modismo → Es Modismo (Sí/No)
                            Modelo  Errores  Válidos Válidos/Total % Válidos
              amazon/nova-micro-v1        1     6532     6532/6533    100.0%
                   microsoft/phi-4       10     6523     6523/6533     99.8%
               amazon/nova-lite-v1        3     6530     6530/6533    100.0%
          cohere/command-r-08-2024        3     6530     6530/6533    100.0%
        qwen/qwen-2.5-72b-instruct      708     5825     5825/6533     89.2%
             google/gemma-2-27b-it       15     6518     6518/6533     99.8%
 meta-llama/llama-3.3-70b-instruct      390     6143     6143/6533     94.0%
        microsoft/wizardlm-2-8x22b      184     6349     6349/6533     97.2%
       meta-llama/llama-4-maverick        8     6525     6525/6533     99.9%
 qwen/qwen2.5-vl-32b-instruct:free     6533        0        0/6533      0.0%
             x-ai/grok-3-mini-beta      284     6249     6249/6533     95.7%
                  perplexity/s

## PROMPT 2: Modismo → Definición

Procesa las respuestas del Prompt 2 y genera datos para métricas comparando las definiciones generadas con el ground truth.

In [18]:
print("=" * 60)
print("PROCESANDO PROMPT 2: Modismo → Definición")
print("=" * 60)

prompt_2_responses = load_prompt_responses(os.path.join(STRAICO_DIR, 'Prompt 2'))

# Procesar datos
prompt_2_data = []
errors_count = 0
skipped_models = []
excluded_models = []  # Modelos excluidos por alto % de errores

for model in DEFAULT_MODELS:
    if model not in prompt_2_responses:
        print(f"⚠ Advertencia: Modelo {model} no encontrado en Prompt 2")
        skipped_models.append(model)
        continue
    
    model_responses = prompt_2_responses[model]
    total_entries = len(model_responses)
    model_errors = 0
    model_data = []  # Datos temporales del modelo
    
    for entry in model_responses:
        modismo = entry.get('modismo', '')
        response = entry.get('response', {})
        
        # Verificar si hay error en la respuesta
        if 'error' in response:
            model_errors += 1
            continue
        
        # Extraer definición generada del output
        definicion_generada = extract_output_field(response, 'definicion')
        
        # Buscar en ground truth
        gt = ground_truth.get(modismo, {})
        definicion_real = gt.get('significado', '')
        
        # Solo agregar si se extrajo la definición correctamente
        if definicion_generada:
            model_data.append({
                'modismo': modismo,
                'definicion_real': definicion_real,
                'modelo': model,
                'definicion_generada': definicion_generada
            })
        else:
            model_errors += 1
    
    # Calcular porcentaje de errores
    error_percentage = (model_errors / total_entries * 100) if total_entries > 0 else 0
    
    # Solo incluir si el porcentaje de errores es <= 50%
    if error_percentage > 50:
        print(f"  {model}: EXCLUIDO ({model_errors}/{total_entries} errores = {error_percentage:.1f}%)")
        excluded_models.append(model)
        errors_count += model_errors
    else:
        # Agregar datos del modelo
        prompt_2_data.extend(model_data)
        errors_count += model_errors
        if model_errors > 0:
            print(f"  {model}: {model_errors} errores omitidos ({error_percentage:.1f}%)")

# Guardar
output_path = os.path.join(OUTPUT_DIR, 'prompt_2_metrics_data.json')
save_json(output_path, prompt_2_data)

print(f"\n✓ Procesados {len(prompt_2_data)} registros válidos")
print(f"✗ Omitidos {errors_count} registros con errores")
if excluded_models:
    print(f"⊗ Modelos excluidos (>50% errores): {', '.join(excluded_models)}")
if skipped_models:
    print(f"⚠ Modelos no encontrados: {', '.join(skipped_models)}")
print()


PROCESANDO PROMPT 2: Modismo → Definición
  amazon/nova-micro-v1: 12 errores omitidos (0.2%)
  microsoft/phi-4: 381 errores omitidos (5.8%)
  amazon/nova-lite-v1: 75 errores omitidos (1.1%)
  cohere/command-r-08-2024: 357 errores omitidos (5.5%)
  qwen/qwen-2.5-72b-instruct: 733 errores omitidos (11.2%)
  google/gemma-2-27b-it: 153 errores omitidos (2.3%)
  meta-llama/llama-3.3-70b-instruct: 378 errores omitidos (5.8%)
  microsoft/wizardlm-2-8x22b: 150 errores omitidos (2.3%)
  meta-llama/llama-4-maverick: 30 errores omitidos (0.5%)
  qwen/qwen2.5-vl-32b-instruct:free: EXCLUIDO (6533/6533 errores = 100.0%)
  x-ai/grok-3-mini-beta: 280 errores omitidos (4.3%)
  perplexity/sonar: 354 errores omitidos (5.4%)
  mistralai/mistral-medium-3: 287 errores omitidos (4.4%)
  mistralai/mixtral-8x7b-instruct: 299 errores omitidos (4.6%)
  google/gemini-2.5-flash: 287 errores omitidos (4.4%)
  meta-llama/llama-3.1-405b-instruct: 702 errores omitidos (10.7%)
  deepseek/deepseek-chat-v3.1: 24 errores 

In [19]:
# Tabla de resultados por modelo para Prompt 2
prompt_2_stats = []

for model in DEFAULT_MODELS:
    if model not in prompt_2_responses:
        continue
    
    model_responses = prompt_2_responses[model]
    total_entries = len(model_responses)
    model_errors = 0
    valid_entries = 0
    
    for entry in model_responses:
        response = entry.get('response', {})
        
        if 'error' in response:
            model_errors += 1
            continue
        
        definicion_generada = extract_output_field(response, 'definicion')
        
        if definicion_generada:
            valid_entries += 1
        else:
            model_errors += 1
    
    prompt_2_stats.append({
        'Modelo': model,
        'Errores': model_errors,
        'Válidos': valid_entries,
        'Válidos/Total': f"{valid_entries}/{total_entries}",
        '% Válidos': f"{(valid_entries/total_entries*100):.1f}%" if total_entries > 0 else "0.0%"
    })

# Crear DataFrame y mostrar
df_prompt_2 = pd.DataFrame(prompt_2_stats)
print("\nTABLA PROMPT 2: Modismo → Definición")
print("=" * 80)
print(df_prompt_2.to_string(index=False))
print("=" * 80)


TABLA PROMPT 2: Modismo → Definición
                            Modelo  Errores  Válidos Válidos/Total % Válidos
              amazon/nova-micro-v1       12     6521     6521/6533     99.8%
                   microsoft/phi-4      381     6152     6152/6533     94.2%
               amazon/nova-lite-v1       75     6458     6458/6533     98.9%
          cohere/command-r-08-2024      357     6176     6176/6533     94.5%
        qwen/qwen-2.5-72b-instruct      733     5800     5800/6533     88.8%
             google/gemma-2-27b-it      153     6380     6380/6533     97.7%
 meta-llama/llama-3.3-70b-instruct      378     6155     6155/6533     94.2%
        microsoft/wizardlm-2-8x22b      150     6383     6383/6533     97.7%
       meta-llama/llama-4-maverick       30     6503     6503/6533     99.5%
 qwen/qwen2.5-vl-32b-instruct:free     6533        0        0/6533      0.0%
             x-ai/grok-3-mini-beta      280     6253     6253/6533     95.7%
                  perplexity/sonar    

## PROMPT 3: Modismo + Ejemplo → Literal + Definición

Procesa las respuestas del Prompt 3 para evaluar la capacidad de los modelos de generar interpretaciones literales.

In [20]:
# Cargar dataset
ground_truth = load_ground_truth('../DataSet/DataSet_ConEjemplos.json')

Dataset cargado: 4897 modismos únicos


In [21]:
print("=" * 60)
print("PROCESANDO PROMPT 3: Modismo + Ejemplo → Literal + Definición")
print("=" * 60)

# Cargar respuestas
prompt_3_responses = load_prompt_responses(os.path.join(STRAICO_DIR, 'Prompt 3'))

# Procesar datos
prompt_3_data = []
errors_count = 0
skipped_models = []
excluded_models = []  # Modelos excluidos por alto % de errores

for model in DEFAULT_MODELS:
    if model not in prompt_3_responses:
        print(f"⚠ Advertencia: Modelo {model} no encontrado en Prompt 3")
        skipped_models.append(model)
        continue
    
    model_responses = prompt_3_responses[model]
    total_entries = len(model_responses)
    model_errors = 0
    model_data = []  # Datos temporales del modelo
    
    for entry in model_responses:
        modismo = entry.get('modismo', '')
        ejemplo = entry.get('ejemplo', '')
        response = entry.get('response', {})
        
        # Verificar que response sea un diccionario
        if not isinstance(response, dict):
            model_errors += 1
            continue
        
        # Verificar si hay error en la respuesta
        if 'error' in response:
            model_errors += 1
            continue
        
        # NOTA: El campo es 'sinonimo' no 'literal' en Prompt 3
        literal_generado = extract_output_field(response, 'sinonimo')
        definicion_generada = extract_output_field(response, 'definicion')
        
        # Buscar en ground truth
        gt = ground_truth.get(modismo, {})
        
        # Solo agregar si se extrajo al menos un campo correctamente
        if literal_generado or definicion_generada:
            model_data.append({
                'modismo': modismo,
                'ejemplo': ejemplo,
                'significado_real': gt.get('significado', ''),
                'modelo': model,
                'literal_generado': literal_generado,
                'definicion_generada': definicion_generada
            })
        else:
            model_errors += 1
    
    # Calcular porcentaje de errores
    error_percentage = (model_errors / total_entries * 100) if total_entries > 0 else 0
    
    # Solo incluir si el porcentaje de errores es <= 50%
    if error_percentage > 50:
        print(f"  {model}: EXCLUIDO ({model_errors}/{total_entries} errores = {error_percentage:.1f}%)")
        excluded_models.append(model)
        errors_count += model_errors
    else:
        # Agregar datos del modelo
        prompt_3_data.extend(model_data)
        errors_count += model_errors
        if model_errors > 0:
            print(f"  {model}: {model_errors} errores omitidos ({error_percentage:.1f}%)")

# Guardar
output_path = os.path.join(OUTPUT_DIR, 'prompt_3_metrics_data.json')
save_json(output_path, prompt_3_data)

print(f"\n✓ Procesados {len(prompt_3_data)} registros válidos")
print(f"✗ Omitidos {errors_count} registros con errores")
if excluded_models:
    print(f"⊗ Modelos excluidos (>50% errores): {', '.join(excluded_models)}")
if skipped_models:
    print(f"⚠ Modelos no encontrados: {', '.join(skipped_models)}")
print()


PROCESANDO PROMPT 3: Modismo + Ejemplo → Literal + Definición
  amazon/nova-micro-v1: 40 errores omitidos (0.8%)
  microsoft/phi-4: 24 errores omitidos (0.5%)
  amazon/nova-lite-v1: 20 errores omitidos (0.4%)
  cohere/command-r-08-2024: 24 errores omitidos (0.5%)
  qwen/qwen-2.5-72b-instruct: 568 errores omitidos (11.6%)
  google/gemma-2-27b-it: 84 errores omitidos (1.7%)
  meta-llama/llama-3.3-70b-instruct: 114 errores omitidos (2.3%)
  microsoft/wizardlm-2-8x22b: EXCLUIDO (4895/4897 errores = 100.0%)
  meta-llama/llama-4-maverick: 28 errores omitidos (0.6%)
  qwen/qwen2.5-vl-32b-instruct:free: EXCLUIDO (4897/4897 errores = 100.0%)
  x-ai/grok-3-mini-beta: 291 errores omitidos (5.9%)
  perplexity/sonar: 3 errores omitidos (0.1%)
  mistralai/mistral-medium-3: 154 errores omitidos (3.1%)
  mistralai/mixtral-8x7b-instruct: 395 errores omitidos (8.1%)
  google/gemini-2.5-flash: 295 errores omitidos (6.0%)
  meta-llama/llama-3.1-405b-instruct: 406 errores omitidos (8.3%)
  deepseek/deepsee

In [22]:
# Tabla de resultados por modelo para Prompt 3
prompt_3_stats = []

for model in DEFAULT_MODELS:
    if model not in prompt_3_responses:
        continue
    
    model_responses = prompt_3_responses[model]
    total_entries = len(model_responses)
    model_errors = 0
    valid_entries = 0
    
    for entry in model_responses:
        response = entry.get('response', {})
        
        if not isinstance(response, dict):
            model_errors += 1
            continue
        
        if 'error' in response:
            model_errors += 1
            continue
        
        literal_generado = extract_output_field(response, 'sinonimo')
        definicion_generada = extract_output_field(response, 'definicion')
        
        if literal_generado or definicion_generada:
            valid_entries += 1
        else:
            model_errors += 1
    
    prompt_3_stats.append({
        'Modelo': model,
        'Errores': model_errors,
        'Válidos': valid_entries,
        'Válidos/Total': f"{valid_entries}/{total_entries}",
        '% Válidos': f"{(valid_entries/total_entries*100):.1f}%" if total_entries > 0 else "0.0%"
    })

# Crear DataFrame y mostrar
df_prompt_3 = pd.DataFrame(prompt_3_stats)
print("\nTABLA PROMPT 3: Modismo + Ejemplo → Literal + Definición")
print("=" * 80)
print(df_prompt_3.to_string(index=False))
print("=" * 80)


TABLA PROMPT 3: Modismo + Ejemplo → Literal + Definición
                            Modelo  Errores  Válidos Válidos/Total % Válidos
              amazon/nova-micro-v1       40     4857     4857/4897     99.2%
                   microsoft/phi-4       24     4873     4873/4897     99.5%
               amazon/nova-lite-v1       20     4877     4877/4897     99.6%
          cohere/command-r-08-2024       24     4873     4873/4897     99.5%
        qwen/qwen-2.5-72b-instruct      568     4329     4329/4897     88.4%
             google/gemma-2-27b-it       84     4813     4813/4897     98.3%
 meta-llama/llama-3.3-70b-instruct      114     4783     4783/4897     97.7%
        microsoft/wizardlm-2-8x22b     4895        2        2/4897      0.0%
       meta-llama/llama-4-maverick       28     4869     4869/4897     99.4%
 qwen/qwen2.5-vl-32b-instruct:free     4897        0        0/4897      0.0%
             x-ai/grok-3-mini-beta      291     4606     4606/4897     94.1%
                  

## Resumen de Archivos Generados

In [23]:
print("=" * 60)
print("RESUMEN DE ARCHIVOS GENERADOS")
print("=" * 60)

import glob

# Listar todos los archivos JSON generados
json_files = glob.glob(os.path.join(OUTPUT_DIR, '*.json'))
json_files.sort()

total_registros = 0
for filepath in json_files:
    filename = os.path.basename(filepath)
    
    with open(filepath, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    num_registros = len(data)
    total_registros += num_registros
    
    print(f"  {filename}: {num_registros} registros")

print("=" * 60)
print(f"Total: {len(json_files)} archivos, {total_registros} registros")
print(f"Ubicación: {OUTPUT_DIR}/")
print("\n✓ Procesamiento completado")

RESUMEN DE ARCHIVOS GENERADOS
  prompt_1_metrics_data.json: 140310 registros
  prompt_2_metrics_data.json: 136369 registros
  prompt_3_metrics_data.json: 99507 registros
Total: 3 archivos, 376186 registros
Ubicación: Clean/

✓ Procesamiento completado
